In [188]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [189]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [190]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    
  def build(self, input_shape):
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, int(input_shape[1])]),trainable=False, name= self.name + "_activation")
    
  def call(self, input):
    output = tf.reshape(input, [1, 16])
    self.a.assign(output)
    return self.a.value()

In [191]:
class Classic_layer_Semeion(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(Classic_layer_Semeion, self).__init__(name = name)
    self.units = 15
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[1]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
    # i pesi dei neuroni sono considerati in colonna
    net = tf.matmul(inputs,self.w)
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()


---

# CUSTOM MODEL

In [192]:
class MQ_Fuzzy_inverter(tf.keras.Model):
  def __init__(self, learning_rate, bias_value, initial_temperature):
    super(MQ_Fuzzy_inverter, self).__init__(name='MQ_Fuzzy_inverter')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = Classic_layer_Semeion(name = "HiddenLayer",weights_init_val=0.1, bias_init_val=bias_value)
    self.o = Classic_layer_Semeion(name = "OutputLayer",weights_init_val=0.1, bias_init_val=bias_value)
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.temperature = tf.Variable(initial_value = initial_temperature,trainable=False, name = "Temperature")
    self.alpha = learning_rate

  def call(self, data):
    input, target = data
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    self.error.assign(tf.math.reduce_sum(tf.math.square(target - res_output)))
    return self.error.value()
  
  def train_step(self, data):
    input, target = data
    for i in tf.range(0, tf.shape(data[0])[0]):
      single_input = tf.reshape(input[i], [1, 16])
      single_target = tf.reshape(target[i], [1, 15])

      ### CALL FOWARD PASS
      self.error.assign(self((single_input, single_target), training = True))

      ### BACK PROPAGATION
        # Update outputs weights
      delta_Ao = self.o.a * (single_target - self.o.a) * (1 - self.o.a)
      delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao) 
      self.o.deltaA.assign(delta_Ao)

        ## Update hidden weights
      delta_Ah = self.h.a * (1 - self.h.a) * tf.matmul(self.o.deltaA, self.o.w)
      delta_Wh = self.alpha * tf.matmul(tf.transpose(self.input_layer.a),delta_Ah) #self.alpha * delta_Ah * self.input_layer.a  # <-- ERROR HERE
      self.h.deltaA.assign(delta_Ah)

        # TO BE UPDATED AFTER TOGETHER
      self.o.w.assign_add(delta_Wo)
      self.h.w.assign_add(delta_Wh)

      # Compute temperature
      #self.temperature.assign(1 - (1 / (1 + self.error) ))
    
    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value()),
        "Hidden activations": tf.squeeze(self.h.a.value()),
        "Output activations": tf.squeeze(self.o.a.value())
        }

# DEFINE PARAMETERS AND INPUT

In [193]:
soggetto = "soggetto_8"

In [194]:
# Model parameters
learning_rate = 1.0
global_bias_value = 0.0001 # this value is used to initialize the bias of all the layers
epochs = 1000
initial_temperature = 1.0
# INITIAL WEIGHT AND TF FUNCTIONTS AND GLOBAL ERROR

In [195]:
# Model input
df_input = pd.read_csv(soggetto + "_input.csv")
df_input = df_input.drop(columns=['SIGMA','bit_type'])
# Model target
df_target = pd.read_csv(soggetto + "_targets.csv")

input = df_input.to_numpy()
targets = df_target.to_numpy()

In [196]:
# Construct an instance of CustomModel
model = MQ_Fuzzy_inverter(learning_rate = learning_rate, bias_value = global_bias_value, initial_temperature = initial_temperature)

In [197]:
model.compile(run_eagerly = True)
model_history = model.fit(x = input, y = targets, epochs=epochs, shuffle=False, verbose = 0)

# Analysis of the results

In [198]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights,Hidden activations,Output activations
0,1.004521,1.0,"[[0.068012804, 0.070463225, 0.084583744, 0.092...","[[-0.044740777, -0.026549269, -0.03869934, -0....","[0.5803435, 0.5759792, 0.58496916, 0.5898052, ...","[0.4574039, 0.4415158, 0.42680353, 0.39741647,..."
1,0.981838,1.0,"[[0.047744714, 0.054600954, 0.07205261, 0.0936...","[[-0.04838515, -0.03031994, -0.040328756, -0.0...","[0.5631674, 0.5566012, 0.56846607, 0.5843372, ...","[0.4522838, 0.43383116, 0.42620498, 0.3955375,..."
2,0.960209,1.0,"[[0.025362467, 0.038103838, 0.05935691, 0.0925...","[[-0.051719166, -0.03561507, -0.043159895, -0....","[0.5437079, 0.5372115, 0.5522334, 0.5767302, 0...","[0.44681907, 0.4266223, 0.42525953, 0.3935544,..."
3,0.938787,1.0,"[[0.0007963666, 0.021101758, 0.046433873, 0.08...","[[-0.055266224, -0.04254543, -0.04744016, -0.0...","[0.52197176, 0.518094, 0.5363465, 0.56696653, ...","[0.4410918, 0.41976118, 0.42369395, 0.39133626..."
4,0.917715,1.0,"[[-0.025981085, 0.003389834, 0.03290877, 0.083...","[[-0.05984587, -0.051174268, -0.053395867, -0....","[0.49813604, 0.49912718, 0.52049315, 0.5551619...","[0.43519753, 0.41332766, 0.4215206, 0.3890508,..."
...,...,...,...,...,...,...
995,0.024118,1.0,"[[-4.7225695, -6.8245893, -6.1956186, -0.79881...","[[-1.627022, -1.9841926, -0.119460806, -0.8598...","[0.14062771, 0.17671987, 0.058885872, 0.009001...","[0.2831738, 0.44139862, 0.3834968, 0.45162436,..."
996,0.024140,1.0,"[[-4.7293954, -6.8281283, -6.199117, -0.797618...","[[-1.626478, -1.9869133, -0.117448054, -0.8559...","[0.14067872, 0.17696491, 0.058760576, 0.008992...","[0.28338584, 0.4415515, 0.38344088, 0.45152295..."
997,0.024163,1.0,"[[-4.736199, -6.8316984, -6.20263, -0.7963983,...","[[-1.625909, -1.9896349, -0.115451545, -0.8519...","[0.14072834, 0.17720518, 0.058637056, 0.008983...","[0.2835959, 0.44171035, 0.3833902, 0.45142466,..."
998,0.024186,1.0,"[[-4.7429786, -6.835297, -6.206154, -0.7951590...","[[-1.6253142, -1.992357, -0.11347104, -0.84794...","[0.14077643, 0.17744078, 0.058515456, 0.008974...","[0.28380388, 0.4418742, 0.3833466, 0.4513299, ..."


In [199]:
fig = px.line(df_model_history[["Error"]])
fig.update_layout(title="Error",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
#fig.write_html("Error and temperature.html")
#fig.write_image("error_temperature.png")
fig.show()

In [200]:
# # create a dictionary with the weights, each key is w in the form w_h_i_j
# weights_hidden = {}
# for i in range(0, len(df_model_history["Hidden weights"][0])):
#   for j in range(0, len(df_model_history["Hidden weights"][0][i])):
#     weights_hidden["w_h_" + str(i+1) + "_" + str(j+1)] = df_model_history["Hidden weights"].apply(lambda x: x[i][j])
# 
# # create a dictionary with the weights, each key is w in the form w_o_i_j
# weights_output = {}
# for i in range(0, len(df_model_history["Output weights"][0])):
#     for j in range(0, len(df_model_history["Output weights"][0][i])):
#         weights_output["w_o_" + str(i+1) + "_" + str(j+1)] = df_model_history["Output weights"].apply(lambda x: x[i][j])


In [201]:
# # create a dataframe with the weights_hidden and weights_output
# df_weights = pd.DataFrame(weights_hidden)
# df_weights = df_weights.join(pd.DataFrame(weights_output))
# df_weights

In [202]:
# df_weights.to_csv("weights.csv")

In [203]:
# # create a dictionary with the activations, each key is a in the form a_h_i
# activations_hidden = {}
# for i in range(0, len(df_model_history["Hidden activations"][0])):
#     activations_hidden["a_h_" + str(i+1)] = df_model_history["Hidden activations"].apply(lambda x: x[i])
# 
# # create a dictionary with the activations, each key is a in the form a_o_i
# activations_output = {}
# for i in range(0, len(df_model_history["Output activations"][0])):
#     activations_output["a_o_" + str(i+1)] = df_model_history["Output activations"].apply(lambda x: x[i])

In [204]:
# # create a dataframe with the activations_hidden and activations_output
# df_activations = pd.DataFrame(activations_hidden)
# df_activations = df_activations.join(pd.DataFrame(activations_output))
# df_activations

In [205]:
# df_activations.to_csv("activations.csv")

In [206]:
# # create a plot displaying the neural network hidden weights over time (epochs) using plotly
# fig1 = go.Figure()
# for i in range(0, len(df_model_history["Hidden weights"][0])):
#     for j in range(0, len(df_model_history["Hidden weights"][0][i])):
#         fig1.add_trace(go.Scatter(x=df_model_history.index, y=df_model_history["Hidden weights"].apply(lambda x: x[i][j]), name="w_h_" + str(i+1) + "_" + str(j+1)))
# fig1.update_layout(height=500, width=800, title_text="Hidden Weights over time")
# fig1.update_yaxes(range=[-1, 1])
# fig1.update_layout(legend_title_text="Click to hide/show", legend_title_font=dict(size=16, family="sans-serif", color="black"))
# fig1.update_traces(line_color='blue')
# fig1.update_traces(hovertemplate=None)
# fig1.update_traces(hoverlabel=dict(font_size=18, font_family="Rockwell"))
# fig1.update_traces(hovertemplate="<b>Epoch</b>: %{x}<br><b>Weight value</b>: %{y}")
# fig1.show()
# # create a plot displaying the neural network output weights over time (epochs) using plotly
# fig2 = go.Figure()
# for i in range(0, len(df_model_history["Output weights"][0])):
#     for j in range(0, len(df_model_history["Output weights"][0][i])):
#         fig2.add_trace(go.Scatter (x=df_model_history.index, y=df_model_history["Output weights"].apply(lambda x: x[i][j]), name="w_o_" + str(i+1) + "_" + str(j+1)))
# fig2.update_layout(height=500, width=800, title_text="Output Weights over time")
# fig2.update_yaxes(range=[-1, 1])
# fig2.update_layout(legend_title_text="Click to hide/show", legend_title_font=dict(size=16, family="sans-serif", color="black"))
# fig2.update_traces(line_color='lightblue')
# fig2.update_traces(hovertemplate=None)
# fig2.update_traces(hoverlabel=dict(font_size=18, font_family="Rockwell"))
# fig2.update_traces(hovertemplate="<b>Epoch</b>: %{x}<br><b>Weight value</b>: %{y}")
# fig2.show()
# # create a plot displaying the neural network hidden activations over time (epochs) using plotly
# fig3 = go.Figure()
# for i in range(0, len(df_model_history["Hidden activations"][0])):
#     fig3.add_trace(go.Scatter(x=df_model_history.index, y=df_model_history["Hidden activations"].apply(lambda x: x[i]), name="a_h_" + str(i+1)))
# fig3.update_layout(height=500, width=800, title_text="Hidden Activations over time")
# fig3.update_yaxes(range=[0, 1])
# fig3.update_layout(legend_title_text="Click to hide/show", legend_title_font=dict(size=16, family="sans-serif", color="black"))
# fig3.update_traces(line_color='red')
# fig3.update_traces(hovertemplate=None)
# fig3.update_traces(hoverlabel=dict(font_size=18, font_family="Rockwell"))
# fig3.update_traces(hovertemplate="<b>Epoch</b>: %{x}<br><b>Weight value</b>: %{y}")
# fig3.show()
# # create a plot displaying the neural network output activations over time (epochs) using plotly
# fig4 = go.Figure()
# for i in range(0, len(df_model_history["Output activations"][0])):
#     fig4.add_trace(go.Scatter(x=df_model_history.index, y=df_model_history["Output activations"].apply(lambda x: x[i]), name="a_o_" + str(i+1)))
# fig4.update_layout(height=500, width=800, title_text="Output Activations over time")
# fig4.update_yaxes(range=[0, 1])
# fig4.update_layout(legend_title_text="Click to hide/show", legend_title_font=dict(size=16, family="sans-serif", color="black"))
# fig4.update_traces(line_color='orange')
# fig4.update_traces(hovertemplate=None)
# fig4.update_traces(hoverlabel=dict(font_size=18, font_family="Rockwell"))
# fig4.update_traces(hovertemplate="<b>Epoch</b>: %{x}<br><b>Weight value</b>: %{y}")
# fig4.show()


In [207]:
# df_model_history.to_csv("model_history.csv")

---